# Testing Patterns / Principles

## TDD Principles


- Software development practice that focuses on writing tests before writing the actual code.
  
- Red → Green → Refactor
  
- Red: Write a failing test that defines a small piece of desired functionality
  - This confirms that the new feature or behavior doesn’t yet exist.

- Green: Write just enough code to make the test pass.
  - Don’t worry about perfection yet—just make the test succeed.

- Refactor: Improve the existing code structure without changing behavior.
  - Clean up duplication, improve readability, and ensure maintainability.

## AAAC: Arrange-Act-Assert-Cleanup

- Arrange
  - where we prepare everything for our test. It’s lining up the dominoes so that the act can do its thing in one, state-changing step. 
  - Preparing objects, starting/killing services, entering records into a database, or even things like defining a URL to query, generating some credentials for a user that doesn’t exist yet, or just waiting for some process to finish.
- Act
  - singular, state-changing action that kicks off the behavior we want to test.
  - behavior is what carries out the changing of the state of the system under test (SUT), and it’s the resulting changed state that we can look at to make a judgement about the behavior. 
  - typically takes the form of a function/method call.
- Assert
  - we look at that resulting state and check if it looks how we’d expect after the dust has settled.
  - where we gather evidence to say the behavior does or does not align with what we expect. 
  - The assert in our test is where we take that measurement/observation and apply our judgement to it.
- Cleanup
  - where the test picks up after itself, so other tests aren’t being accidentally influenced by it.

- Keep each phase visually separated (blank lines help)
- Assert invariants (shape, monotonicity, conservation) not just values

In [ ]:
def normalize(xs):
    total = sum(xs)
    return [x/total for x in xs] if total else xs

# Arrange
xs = [2, 2, 6]
# Act
ys = normalize(xs)
# Assert
assert abs(sum(ys) - 1.0) < 1e-9
assert len(ys) == len(xs)

## FIRST Principles

- A set of guidelines to make tests effective and reliable

| Letter | Principle      | Meaning                                                        |
|--------|---------------|----------------------------------------------------------------|
| F      | Fast          | Tests should run quickly to encourage frequent runs.            |
| I      | Independent   | Tests should not depend on each other.                          |
| R      | Repeatable    | Tests should produce the same result every time.                |
| S      | Self-Validating | Tests should have clear pass/fail outcomes (no print statements). |
| T      | Timely        | Write tests at the right time — before or alongside code.       |

## Isolation Principle

- Each test should verify one unit of behavior in isolation.

In [ ]:
# Isolation Principle Example:
# Each test should verify one unit of behavior in isolation, without relying on shared state.

def increment(x):
    return x + 1

def test_increment_isolated():
    # Arrange
    value = 3
    # Act
    result = increment(value)
    # Assert
    assert result == 4

def test_increment_does_not_affect_other():
    # Arrange
    value = 10
    # Act
    result = increment(value)
    # Assert
    assert result == 11

print("Both tests are isolated and do not depend on each other's state.")

## Parametrized Testing

- run the same test logic multiple times with different inputs and expected outputs, without duplicating code.

In [2]:
import pytest

@pytest.mark.parametrize("a,b,expected", [(1,2,3), (0,0,0), (-1,1,0)])
def test_add(a,b,expected):
    assert a+b == expected

## Behavior-Driven Testing (BDD)

## Testing Doubles

- stand-ins for real objects used in testing to isolate the system under test (SUT). 
- They help simulate specific scenarios, avoid side effects (like network or database calls), and make tests deterministic.

| Type   | Description                                                                 | Example                                      |
|--------|-----------------------------------------------------------------------------|----------------------------------------------|
| Dummy  | Passed around but never actually used; only fills parameter lists.           | `DummyLogger()` with no functionality.       |
| Stub   | Provides predefined responses to method calls; used to control test conditions. | A stub database that always returns a fixed user object. |
| Fake   | Has working implementation but is simplified or not production-ready.        | In-memory database replacing a real one.     |
| Spy    | Records information about how it was called (e.g., method name, arguments).  | Verifying that an email-sending function was called once. |
| Mock   | Pre-programmed expectations about how it should be used; fails test if expectations aren’t met. | Mock API client expecting `.get("/user/1")` to be called exactly once. |

### Test Organization Patterns

In [ ]:
# Structure large test suites for maintainability:
# tests/
#   ├── test_unit/
#   │     ├── test_math.py
#   │     └── test_utils.py
#   ├── test_integration/
#   │     └── test_api_endpoints.py
#   ├── conftest.py   # pytest fixtures available project-wide
#   └── data/         # golden files, mock data
print("Organize tests by type and use fixtures for shared setup.")

# Testing Types

# pytest

### Assertions

In [ ]:
def mean(xs):
    assert len(xs) > 0, "mean() requires non-empty list"
    return sum(xs) / len(xs)

print(mean([1,2,3]))
# mean([])  # would raise AssertionError

### Testing Hooks

In [ ]:
def is_pal(s: str) -> bool:
    """
    Return True if s is a palindrome.

    Examples:
    >>> is_pal('racecar')
    True
    >>> is_pal('abc')
    False
    """
    return s == s[::-1]

assert is_pal("madam") is True
assert is_pal("nope") is False
# To run doctests in a script:
# if __name__ == "__main__":
#     import doctest; doctest.testmod()

### Unit Testing

In [ ]:
# Pytest discovers functions named test_* in files/test modules.
# Example tests (showing the style—run with `pytest -q` in a terminal):

def add(a, b): return a + b

def test_add_basic():
    assert add(2, 3) == 5

### Parametrized Tests

In [ ]:
# Test multiple inputs with one function (concept).
#
# import pytest
# @pytest.mark.parametrize("nums,expected", [
#     ([1,2,3], 6),
#     ([0,0,0], 0),
#     ([-1,1], 0),
# ])
# def test_sum(nums, expected):
#     assert sum(nums) == expected
print("Use pytest.mark.parametrize for testing many cases compactly.")

### Hypothesis testing

In [ ]:
# If Hypothesis is installed: `pip install hypothesis`
# from hypothesis import given, strategies as st
# @given(st.lists(st.integers()))
# def test_reverse_reverse(xs):
#     ys = list(reversed(list(reversed(xs))))
#     assert ys == xs
print("Property-based testing checks invariants across many random inputs.")

### Fuzz Testing

In [ ]:
import random

def reverse_twice(xs):
    return list(reversed(list(reversed(xs))))

for _ in range(5):
    seq = [random.randint(-10,10) for _ in range(5)]
    assert reverse_twice(seq) == seq

print("Fuzz tests run random inputs to catch rare bugs.")

### Snapshot / Golden File Testing

In [ ]:
import json, tempfile, os

def serialize(data):
    return json.dumps(data, indent=2, sort_keys=True)

expected_snapshot = '{"a": 1, "b": 2}'
snapshot = serialize({"a": 1, "b": 2})

assert snapshot.strip() == expected_snapshot.strip()
print("Snapshot test passed.")

### Monkeypatching (temporary overrides)

In [ ]:
# Example using contextlib for a manual override.
import contextlib

def get_user():
    import os
    return os.getenv("USER", "unknown")

@contextlib.contextmanager
def mock_env(var, val):
    import os
    old = os.environ.get(var)
    os.environ[var] = val
    try:
        yield
    finally:
        if old is None:
            del os.environ[var]
        else:
            os.environ[var] = old

with mock_env("USER", "maverick"):
    assert get_user() == "maverick"

print("Manual monkeypatch for env vars successful.")

### Integration Tests

In [ ]:
# Simulate end-to-end flow using multiple functions together.
def load_data():
    return [1, 2, 3]

def process_data(xs):
    return [x * 2 for x in xs]

def main_pipeline():
    return sum(process_data(load_data()))

assert main_pipeline() == 12
print("Integration test successful: pipeline verified.")

### Regression Tests

In [ ]:
# Catch re-introduced bugs by locking in a previously failing case.
def divide(a, b):
    if b == 0:
        raise ZeroDivisionError
    return a / b

def test_divide_regression():
    try:
        divide(1, 0)
    except ZeroDivisionError:
        pass
    else:
        raise AssertionError("Expected ZeroDivisionError")

test_divide_regression()
print("Regression test ensures old bug stays fixed.")

### Fixtures

In [ ]:
# Fixtures help share setup/teardown code cleanly.
# (Example commented so notebook doesn’t error.)
#
# import pytest
#
# @pytest.fixture
# def sample_data():
#     return [1, 2, 3]
#
# def test_sum(sample_data):
#     assert sum(sample_data) == 6
#
# Run with:
# pytest -v
print("pytest fixtures allow shared setup/teardown between tests.")

### Coverage & CLI

In [ ]:
# From terminal:
#   pytest -q
#   pytest -q -k "keyword"          # subset by name
#   pytest -q -x                     # stop after first failure
#   pytest --maxfail=1 --disable-warnings -q
#   coverage run -m pytest && coverage html
print("Run pytest/coverage from terminal; see comments for common commands.")